# Real World Testing

This notebook is used for generating data in Table 5

Color and depth videos, and raw frames are included in `recordings`

In [1]:
import os
os.chdir(f'{os.getcwd()}/../../')
WORKING_DIR = './evaluation/real_world_testing'

In [2]:
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
def gen_report(theta):
    exp_logs = []

    for g in glob.glob(f'{WORKING_DIR}/logs/*.log'):
        df_log = pd.read_csv(g, header=None)
        df_log[[0]] = df_log[[0]].max() - df_log[[0]]
        df_log = df_log.rename(columns={0: 'nFrame'})

        f_name = os.path.basename(g).replace('.log', '')

        df_log[['name']] = f_name
        df_log[['mark']] = 'a'

        exp_logs.append(df_log)
        
        df_log = pd.read_csv(f'{WORKING_DIR}/logs/{theta}/{f_name}.zip.log', header=None)
        df_log[[0]] = df_log[[0]].max() - df_log[[0]]
        df_log = df_log.rename(columns={0: 'nFrame'})
        
        df_log[['name']] = f_name
        df_log[['mark']] = 't'

        exp_logs.append(df_log)
        

    exp_logs = pd.concat(exp_logs).groupby(['name', 'mark'])
    
    report = pd.DataFrame(columns=[
        'Exp Name', 'SHC RMSE Mean', 'SHC RMSE Std', 'SHC RMSE Min', 'SHC RMSE Max', 
        'Num Frames Triggered', 'Num Frames Total', 'Triggering Percentage'
    ])

    for exp_name in ['05_10_2021-19_52_10', '05_10_2021-19_53_46', '05_10_2021-19_55_33']:
        df_all = exp_logs.get_group((exp_name, 'a'))
        df_trigger = exp_logs.get_group((exp_name, 't'))

        triggered_frames = df_trigger[['nFrame']].values.reshape((-1))

        shc_all = df_all[range(1, 28)].values
        shc_triggered = df_trigger[range(1, 28)].values

        idx = np.zeros(shc_all.shape[0], dtype=np.int)
        idx[triggered_frames] = np.arange(len(triggered_frames))

        i_max = 0

        for i in range(len(idx)):
            if idx[i] == 0 and idx[i] < i_max:
                idx[i] = i_max

            if idx[i] > i_max:
                i_max = idx[i]

        shc_rmse = np.sqrt(((shc_all - shc_triggered[idx, :]) ** 2).mean(axis=-1))

        report.loc[len(report)] = [
            exp_name, shc_rmse.mean(), shc_rmse.std(), shc_rmse.min(), shc_rmse.max(),
            shc_triggered.shape[0], shc_all.shape[0], shc_triggered.shape[0] / shc_all.shape[0] * 100
        ]
        
    return report

In [4]:
gen_report(0.5)

,Exp Name,SHC RMSE Mean,SHC RMSE Std,SHC RMSE Min,SHC RMSE Max,Num Frames Triggered,Num Frames Total,Triggering Percentage
0,05_10_2021-19_52_10,0.017995,0.011924,0.0,0.040749,22,1458,1.508916
1,05_10_2021-19_53_46,0.011290,0.006547,0.0,0.027012,22,817,2.692778
2,05_10_2021-19_55_33,0.004046,0.004728,0.0,0.023125,458,946,48.414376


In [5]:
gen_report(0.6)

,Exp Name,SHC RMSE Mean,SHC RMSE Std,SHC RMSE Min,SHC RMSE Max,Num Frames Triggered,Num Frames Total,Triggering Percentage
0,05_10_2021-19_52_10,0.019279,0.017470,0.0,0.055127,4,1458,0.274348
1,05_10_2021-19_53_46,0.011014,0.003809,0.0,0.021699,2,817,0.244798
2,05_10_2021-19_55_33,0.006975,0.005402,0.0,0.026249,224,946,23.678647


In [6]:
gen_report(0.7)

,Exp Name,SHC RMSE Mean,SHC RMSE Std,SHC RMSE Min,SHC RMSE Max,Num Frames Triggered,Num Frames Total,Triggering Percentage
0,05_10_2021-19_52_10,0.021686,0.016918,0.0,0.058962,2,1458,0.137174
1,05_10_2021-19_53_46,0.018001,0.006080,0.0,0.031367,2,817,0.244798
2,05_10_2021-19_55_33,0.010175,0.005094,0.0,0.029528,56,946,5.919662
